# Notebook for the Dutch Dyslexia Programme (DDP) dataset

This notebook contains four sections. 

+ In section 1. [Loading EEG-data and metadata](#1ddp) the EEG-data and metadata is loaded. 
+ In section 2. [The DDP Experiment](#2ddp) the structure of the auditory oddball experiment is explained and visualised. There is an emphasis on the stimulus events. 
+ Section 3. [Processing](#3ddp) is for processing and cleaning up the raw files. The raw data is split into epochs, i.e. partition surrounding the events. These epochs are stored in a .fif file, which is the standard extension for the mne package. 
+ In section 4. [The Event Related Potential (ERP)](#4ddp) (ERPs) that result from the events are explored.

#### Import Packages

In [ ]:
import mne
import numpy as np
import pandas as pd
import os
import glob
import ipywidgets
import wave # For analyzing .wav audio files

from functions import data_io
from functions import processing
from functions import display_helper

from functions.ddp import DDP
ddp = DDP()
from functions.sequences import DDPSequence

import local_paths

<br>
<br>


---
<a id='1ddp'></a>
## 1. Load and sort data and metadata

#### Create labels from raw .cnt files.

+ The label columns are: filename / code / age_group / age_days
+ The 'create_labels' function takes as inut the dataset location, and the directory where the age metadata is stored

In [ ]:
directory_age_metadata = os.path.join(local_paths.DDP_metadata, "ages")
df_raw = ddp.create_labels(local_paths.DDP_dataset, directory_age_metadata)

#### Participants Info

In [ ]:
min_age, max_age = df_raw["age_days"].min(), df_raw["age_days"].max()
print(f"The participants are between the age of {min_age} "
      f"and {int(max_age)} days. "
      f"({round(min_age/30, 1)} to "
      f"{round(max_age/30,1)} months)")

#### Merge raws from same experiment

Many experiments are split into multipe raw files, but it is more useful to have just one file per experiment. This is why the filenames of the raw files are sorted per experiment in the *experiments_paths* below.

+ *experiments_names* contains the new names of each experiment
+ *experiments_paths* contains a list of the paths to the .cnt files for each experiment.

In [ ]:
experiments_names = []
experiments_paths = []

# For each experiment: sort by unique personal_code and age_group
for age_group in df_raw["age_group"].unique():
    df_age_group = df_raw.loc[df_raw["age_group"] == age_group]
    for personal_code in df_age_group["participant"].unique():
        experiments_names.append(f"{int(personal_code)}_{age_group}")
        
        # Get the raw_paths from each experiment
        df_experiment = df_age_group.loc[df_raw["participant"] == personal_code]
        raw_filenames = df_experiment["filename"].tolist()        
        raw_paths = [os.path.join(local_paths.DDP_dataset, f"{age_group}mnd mmn", filename) for filename in raw_filenames]
        experiments_paths.append(raw_paths)
        
print(f"The {len(df_raw.index)} raw files collectively contain {len(experiments_names)} distinct experiments")

#### Choose file with widget:

Loading the raw .cnt gives two RuntimeWarnings. These can be ignored. 

In [ ]:
def f(file):
    raw_paths = experiments_paths[experiments_names.index(file)]
    
    global raw
    raw = ddp.read_raw(raw_paths)
    print(f"The experiment {file} consists of {len(raw_paths)} raw file(s):")
    print(raw_paths)

experiment_paths_widget = ipywidgets.interactive(f, file=experiments_names)
display(experiment_paths_widget)

<br>
<br>


---
<a id='2ddp'></a>

## 2. The DDP Experiment

The DDP experiment is an *auditory oddball experiment*. Children listen to a sequence that contains standard and deviant syllables in order to elicit the *mismatch response*. 
   
For measurement sometimes 64 and sometimes 30 electrodes are used. The measurement frequency is 500 Hz. 
    
The experiment is around __40 minutes__, with sequences of around 10 minutes containing four different conditions. Ideally, each condition has 400 standards /b/ak (events: 2, 3, 4, 5), 50 deviants /d/ak (events: 55, 66, 77, 88), and 50 first standards (events: 12, 13, 14, 15)



#### Load events

In [ ]:
events, event_dict = mne.events_from_annotations(raw)
fig = mne.viz.plot_events(events, event_id = event_dict)

#### Sort events into standards, deviants, and first-standards.
The event_id's of the _standard_ is 1, _deviant_ is 2, _first_standard_ is 3.

In [ ]:
events_3, event_dict_3 = ddp.events_from_raw(raw)
fig = mne.viz.plot_events(events_3, event_id = event_dict_3)

#### Show part of the EEG signal
When a new experiment is chosen, this cell needs to be run again to visualise its EEG data.

In [ ]:
# Makes the plot interactive, comment out if not working:
%matplotlib widget 
fig = mne.viz.plot_raw(raw, events_3, n_channels=3, scalings=200e-6, duration=5, start=1000)

<br>
<br>

---

<a id='3ddp'></a>
## 3. Processing
#### Filtering DDP dataset and rejecting bad trials

The EEG data located in _local_paths.DDP_dataset_ is cleaned with the following techniques:
+ A high-pass filter on the raw EEG sequence with cutoff frequency 0.1 Hz to remove slow trends
+ Splitting the raw data into 1 second epochs in which the event occurs at 0.2s.
+ The epochs are cleaned with the autoreject library. This library contains classes that automatically reject bad trials and repair bad sensors in EEG data. The AutoReject and Ransac classes are used. https://autoreject.github.io/stable/index.html
+ A low-pass filter on the epochs 

The function process_raw splits the raw files up into epochs and saves the events externally. It takes a while to process each file, mainly due to the complexity of the autoreject method. To save some time, multiple raw files are processed simultaniously via multiprocessing.

In [ ]:
# for i in range(len(experiments_names)):
#     process_raw(i, experiments_names, experiments_paths, ddp, local_paths.DDP_epochs)

## Multiprocessing:
processing.process_raw_multiprocess(experiments_names, experiments_paths, ddp, local_paths.DDP_epochs, verbose=False)

#### Extract valid experiments
Processed files with too few standards and deviants are considered invalid.

In [ ]:
valid_experiments = processing.valid_experiments(ddp, local_paths.DDP_epochs_events,
                                                 min_standards=500, 
                                                 min_deviants=80)
experiment_list = list(set(valid_experiments)-set(ddp.wrong_channels_experiments))
print(f"{len(ddp.wrong_channels_experiments)} experiments have incorrect channels. "
      f"{len(experiment_list)} experiments remain")

<br>
<br>

---
<a id='4ddp'></a>
## 4. The Event Related Potential (ERP)

+ The voltage change in the brain as a response to an event is called the *event-related potential* (ERP)
+ The response difference between a standard and deviant ERP is called the *mismatch response* (MMR).
+ The mismatch response can be analysed to predict differences between participants.

#### Choose experiment to analyse

In [ ]:
def load_participant_data(experiment):
    global epochs
    epochs = mne.read_epochs(os.path.join(local_paths.DDP_epochs, experiment + "_epo.fif"), verbose = 0)
    print(f"Loaded experiment: {experiment} ")

ipywidgets.interact(load_participant_data, experiment=valid_experiments);

#### Widget for plotting standard, deviant, and mismatch ERPs

In [ ]:
event_type = ipywidgets.RadioButtons(options=["standard", "deviant", "MMN"], 
                                     description='Event type:', 
                                     value="standard")
def plot_ERP_widget(ev):
    if(ev=="MMN"):
        evoked = mne.combine_evoked([epochs["deviant"].average(), epochs["standard"].average()], weights = [1, -1])
    else:
        evoked = epochs[ev].average()
    fig = evoked.plot(spatial_colors = True)

ui = ipywidgets.HBox([event_type])
out = ipywidgets.interactive_output(plot_ERP_widget, {'ev': event_type})
display(ui, out)

#### Widget for modifying data

In the deep learning notebook, the model uses a sequence of data. This data is show in the plot below. Tweak the widget to see the effect of changing the parameters on an ERP. The plot can take a while to load.

+ *sample_rate*: The number of data points in a second of each channel.
+ *n_trials_averaged*: The number of trials averaged to form the ERP.
+ *gaussian_noise*: The standard deviation of the noise added to each datapoint to reduce overfitting.

In [ ]:
def show_sequence_data(p, f, n, g):
    labels = ddp.create_labels(local_paths.DDP_dataset, directory_age_metadata)
    sequence = DDPSequence(experiment_list, labels, local_paths.DDP_epochs, batch_size=1, 
                               sample_rate=f, n_trials_averaged=n, gaussian_noise=g*1e-6)
    x, y = sequence.__getitem__(p, True)
    
    print(f"The shape of one data instance is {x[0].shape}")
    display_helper.plot_array_as_evoked(x[0], ddp.channels_epod_ddp, frequency=f, n_trials=n)
    print(f"In this experiment the age of the participant is {int(y[0])} days.")

# Widget sliders
participant = ipywidgets.IntSlider(description="participant")
sample_rate = ipywidgets.IntSlider(description="sample_rate", value=2049, min=10, max=2049)
n_trials_averaged = ipywidgets.IntSlider(description="n_trials_averaged", value=30)
gaussian_noise = ipywidgets.FloatSlider(description="gaussian_noise", max=3)
# Widget setup
ui = ipywidgets.HBox([participant, sample_rate, n_trials_averaged, gaussian_noise])
out = ipywidgets.interactive_output(show_sequence_data, {'p':participant, 'f': sample_rate, "n": n_trials_averaged, "g": gaussian_noise})
display(ui, out)

Beware, the absolute value of the y-axis (microVolt) is meaningless due to data normalization. Each data-point is divided by the standard deviation of all the signals.